## An example of using an external JS library in a callback

In [1]:
import numpy as np

import biocircuits

import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
x = np.linspace(1, 5, 200)
y = np.sqrt(x)
p = bokeh.plotting.figure(
    frame_width=300,
    frame_height=200,
    x_axis_label="x",
    y_axis_label="y",
    x_range=[1, 5],
)

cds = bokeh.models.ColumnDataSource(dict(x=x, y=y))
p.line(source=cds, x="x", y="y", line_width=2)

selector = bokeh.models.Select(title="function", value="sqrt", options=["sqrt", "log"])

jscode = """
let fun;
if (selector.value == 'sqrt') fun = math.sqrt;
else fun = math.log;

for (let i = 0; i < cds.data['x'].length; i++) {
    cds.data['y'][i] = fun(cds.data['x'][i])
}

cds.change.emit();
"""

selector.js_on_change(
    "value", bokeh.models.CustomJS(code=jscode, args=dict(selector=selector, cds=cds))
)

# Add external JavaScript library by updating Jinja template
template = biocircuits.viz.addjs_jinja(
    "https://cdnjs.cloudflare.com/ajax/libs/mathjs/11.9.1/math.js"
)

layout = bokeh.layouts.row(
    p,
    bokeh.layouts.row(
        bokeh.models.Spacer(width=30),
        bokeh.layouts.column(bokeh.models.Spacer(height=70), selector),
    ),
)

biocircuits.viz.bokeh_show(layout, filename='sqrt_or_log.html', template=template)